In [2]:
import pandas as pd
import numpy as np

In [ ]:
#opendartreader

In [32]:
pip install OpenDartReader

Note: you may need to restart the kernel to use updated packages.


In [46]:
import OpenDartReader

#API KEY 지정
my_api="8a0ec5ddb6a2466ce809e621b22baad03792b9a2"

# OpenDartReader 객체 생성
dart = OpenDartReader(my_api)


In [49]:
indicators

['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익']

In [100]:
result = dart.finstate('005380', 2022)


In [90]:
result = result.loc[result['fs_nm']=='재무제표']

In [91]:
result = result.loc[result['account_nm'].isin(['당기순이익','영업이익','매출액'])]

In [92]:
result = result.loc[:, ["account_nm","fs_nm","sj_nm","thstrm_dt","thstrm_amount"]]


In [107]:
result[result.account_nm == '당기순이익'][["account_nm","fs_nm","sj_nm","thstrm_dt","thstrm_amount"]]

,account_nm,fs_nm,sj_nm,thstrm_dt,thstrm_amount
24,당기순이익,재무제표,손익계산서,2022.01.01 ~ 2022.12.31,"3,701,958,000,000"


In [52]:
# 사용자 정의 함수 지정 
# find_financial_indicators

# 종목명, 연도, 수집할 지표 목록을 입력받아 주요 재무지표 찾아서 반환하는 함수

def find_financial_indicators(stock_code, year, indicators):
    report = dart.finstate(stock_code, year) # 데이터 가져오기
    if report is None: # 리포트가 없다면 (참고: 리포트가 없으면 None을 반환)
        # 리포트가 없으면, 당기, 전기, 전전기 값 모두 제거
        data =[[stock_code, year] + [np.nan]* len(indicators)]
        return pd.DataFrame(data, columns = ["기업", "연도"] + indicators)
    
    else: 
        report = report[report['account_nm'].isin(indicators)] # 관련 지표로 필터링
        if sum(report['fs_nm'] == "연결재무제표") > 0:
            # 연결재무제표 데이터가 있으면 연결 재무제표를 사용
            report = report.loc[report['fs_nm'] =='연결재무제표']
            
        else:
            # 연결재무제표 데이터가 없으면 일반재무제표를 사용
            report = report.loc[report['fs_nm'] =='재무제표']
            
        data = []
        for y,c in zip([year], ['thstrm_amount','frmtrm_amount', 'bfefrmtrm_amount']):
            record = [stock_code, y]
            for indic in indicators:
                # account_nm이 indic인 행의 c 컬럼 값을 가져옴
                if sum(report['account_nm'] == indic) >0: # 해당 지표가 있다면 추가 (지표가 없는 경우도 있음)
                    value = report.loc[report['account_nm'] == indic, c].iloc[0]
                else : 
                    value = np.nan
                    
                record.append(value)
            
            data.append(record)
        return pd.DataFrame(data, columns =["기업", "연도"] + indicators)
            

**엔씨소프트 036570** 

**넷마블 251270** 

**크래프톤 259960**

**삼성전자 005930** 

**SK하이닉스 000660** 

**DB 하이텍 000990**

**현대차 005380** 

**현대모비스 012330**

**HL 만도 204320**

**기아 000270**

**LG이노텍 011070**

**KT 030200**

**LG에너지솔루션 373220** 

**삼성SDI 006400**

**LG화학 051910**

**SK이노베이션096770**

In [30]:
stock_code = ['036570','251270','259960','005930','000660','000990','005380','012330',
              '204320','000270','011070','030200','373220','006400','051910','096770']

In [21]:
stock_name = ['엔씨소프트','넷마블','크래프톤','삼성전자','SK하이닉스','DB 하이텍','현대차','현대모비스',
             'HL만도','기아','LG이노텍','KT','LG에너지솔루션','삼성SDI','LG화학','SK이노베이션']

In [23]:
stock_list = pd.DataFrame({'stock_code': stock_code, 'stock_name': stock_name})

In [24]:
stock_list

,stock_code,stock_name
0,036570,엔씨소프트
1,251270,넷마블
2,259960,크래프톤
3,005930,삼성전자
4,000660,SK하이닉스
5,000990,DB 하이텍
6,005380,현대차
7,012330,현대모비스
8,204320,HL만도
9,000270,기아


In [71]:
import time
indicators = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익']
data = pd.DataFrame() # 이 데이터프레임에 각각의 데이터를 추가할 예정

for stock in stock_code:
    try: 
        result = find_financial_indicators(stock, 2022, indicators) #재무제표 데이터
    except:
        pass
    data = pd.concat([data,result], axis =0, ignore_index = True)
    time.sleep(1)

In [103]:
data

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
0,036570,2022,"4,437,632,450,781","1,239,083,608,249","3,198,548,842,532","2,571,792,281,800","559,006,120,606","435,990,309,263"
1,251270,2022,"8,935,573,715,860","3,313,765,261,202","5,621,808,454,658","2,673,408,275,832","-108,688,598,689","-886,357,560,393"
2,259960,2022,"6,030,311,640,765","917,461,503,208","5,112,850,137,557",NaN,"751,606,569,542","500,154,032,217"
3,005930,2022,"448,424,507,000,000","93,674,903,000,000","354,749,604,000,000","302,231,360,000,000","43,376,630,000,000","55,654,077,000,000"
4,000660,2022,"103,871,512,000,000","40,580,970,000,000","63,290,542,000,000","44,621,568,000,000","6,809,417,000,000","2,241,669,000,000"
5,000990,2022,"2,122,700,474,808","484,416,396,415","1,638,284,078,393","1,675,287,533,666","768,746,685,740","558,836,662,217"
6,005380,2022,"255,742,462,000,000","164,845,917,000,000","90,896,545,000,000","142,527,535,000,000","9,819,769,000,000",NaN
7,012330,2022,"55,406,698,000,000","17,599,066,000,000","37,807,632,000,000","51,906,293,000,000","2,026,531,000,000","2,487,244,000,000"
8,204320,2022,"5,845,841,407,280","3,572,095,295,017","2,273,746,112,263","7,516,163,841,233","248,102,024,875","118,287,647,376"
9,000270,2022,"73,710,965,000,000","34,367,884,000,000","39,343,081,000,000","86,559,029,000,000","7,233,120,000,000","5,408,976,000,000"


In [104]:
data.to_excel('dart.xlsx')